<a href="https://colab.research.google.com/github/dernameistegal/airbnb_price/blob/main/data_transformations/translate_listings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Translation of airbnb dataset into english

In [1]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


## Preliminary works
- add comments_en column with values None
- add lang column with values None
- replace na values in comments colum with "no text"

In [8]:
%%capture
!wget http://data.insideairbnb.com/austria/vienna/vienna/2021-11-07/data/listings.csv.gz

In [48]:
listings = pd.read_csv("listings.csv.gz")

In [40]:
%%capture
!pip install git+https://github.com/neuml/txtai#egg=txtai[pipeline]

In [72]:
import torch
path = "/content/drive/MyDrive/Colab/airbnb/data/translations/listings"
listings = pd.read_pickle(path)

In [21]:
# todo
listings.description
listings.host_about
listings.name

0                                b&b near Old Danube river
1         central cityapartement- wifi- nice neighbourhood
2                           Near Palace Schönbrunn, Apt. 1
3        little studio- next to citycenter- wifi- nice ...
4                          Flat in the Center with Terrace
                               ...                        
11404                                  New Apartment Top 4
11405                                  New Apartment Top 2
11406                                  New Apartment Top 6
11407      Schöne Wohnung zwischen Augarten und Donaukanal
11408    Private room in Vienna next to Westbahnhof PrG...
Name: name, Length: 11409, dtype: object

In [49]:
listings[["description_en", "host_about_en", "name_en"]] = listings[["description", "host_about", "name"]]
listings["lang"] = listings["description"]
listings[["description_en", "host_about_en", "name_en", "lang"]] = None

In [50]:
listings[["description", "host_about", "name"]] = listings[["description", "host_about", "name"]].fillna("no text")

In [53]:
from torch.utils.data import DataLoader, Dataset

class Translatedata(Dataset):
    def __init__(self, data, column):
        super().__init__()
        self.data = data
        self.column = column

    def __getitem__(self, key):
        return self.data[self.column].iloc[key], self.data.index[key]

    def __len__(self):
        return len(self.data)

In [54]:
dataset = Translatedata(listings, "description")
dataloader = DataLoader(dataset, batch_size=200)

# Detect language and sort by it

In [55]:
# detect language
from txtai.pipeline import Translation
translate = Translation(batch=64)
with torch.no_grad():
    for i, (text, id) in enumerate(dataloader):
        text = list(text)
        try: 
            if listings.loc[id, "lang"].isnull().values.any():
                language = translate.detect(list(text))
                listings.loc[id, "lang"] = language
        except:
            continue
        last_id = id[-1]
        print(i)

Downloading:   0%|          | 0.00/916k [00:00<?, ?B/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57


In [56]:
listings = listings.sort_values(by="lang")

In [61]:
path = "/content/drive/MyDrive/Colab/airbnb/data/translations/listings"
listings.to_pickle(path)

In [60]:
listings["lang"].value_counts()

en    8126
de    3184
es      17
ru      16
fr      14
it      13
zh      11
ko       7
hu       5
uk       4
pl       2
ja       2
sv       2
hr       1
sh       1
tr       1
sl       1
sk       1
ro       1
Name: lang, dtype: int64

# Translate text

In [ ]:
#@title this is script.py
from txtai.pipeline import Translation
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Create translation model
translate = Translation(batch=64)

path = "/content/drive/MyDrive/Colab/airbnb/data/translations/listings"
listings = pd.read_pickle(path)

class Translatedata(Dataset):
    def __init__(self, data, column):
        super().__init__()
        self.data = data
        self.column = column

    def __getitem__(self, key):
        return self.data[self.column].iloc[key], self.data.index[key]

    def __len__(self):
        return len(self.data)

dataset = Translatedata(listings)
dataloader = DataLoader(dataset, batch_size=64)


with torch.no_grad():
    for i, (text, id) in enumerate(dataloader):
        text = list(text)
        if listings.loc[id, Translatedata.column +"_en"].isnull().values.any():

            sourcefinder = listings.loc[id, "lang"].unique()
            if len(sourcefinder) >= 1:
                  print(sourcefinder)
                  sourcefinder = None
            else:
                print(i)
                sourcefinder = sourcefinder[0]

            translation = translate(list(text), "en", source = sourcefinder)
            listings.loc[id, Translatedata.column +"_en"] = translation

        
        listings.to_pickle(path)

In [81]:
# description batch size 50
!python script.py

0
1
2
3
4
5
6
7
8
9
10
Token indices sequence length is longer than the specified maximum sequence length for this model (642 > 512). Running this sequence through the model will result in indexing errors
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
['de' 'en']
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
['en' 'es' 'fr'

In [94]:
listings = pd.read_pickle(path)

In [88]:
path = "/content/drive/MyDrive/Colab/airbnb/data/translations/listings_secure_save"
listings.to_pickle(path)

In [104]:
len(listings["host_id"].unique())

6338

In [113]:
unique_host_about = listings["host_about"].unique()
unique_host_about = pd.DataFrame(unique_host_about, columns=["host_about"])
unique_host_about["host_about_en"] = unique_host_about["host_about"]
unique_host_about["host_about_en"] = None
unique_host_about.to_pickle("/content/drive/MyDrive/Colab/airbnb/data/translations/host_about")

In [ ]:
#@title new script
from txtai.pipeline import Translation
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Create translation model
translate = Translation(batch=64)

path = "/content/drive/MyDrive/Colab/airbnb/data/translations/host_about"
listings = pd.read_pickle(path)

class Translatedata(Dataset):
    def __init__(self, data, column):
        super().__init__()
        self.data = data
        self.column = column

    def __getitem__(self, key):
        return self.data[self.column].iloc[key], self.data.index[key]

    def __len__(self):
        return len(self.data)

dataset = Translatedata(listings, "host_about")
dataloader = DataLoader(dataset, batch_size=64)

jump_value=0
exit_value=1000

with torch.no_grad():
    for i, (text, id) in enumerate(dataloader):
        text = list(text)
        if i < jump_value:
            print("jumped", i)
            continue
        if i > exit_value:
            break
        if listings.loc[id, dataset.column +"_en"].isnull().values.any():
            print(i)
            translation = translate(list(text), "en")
            listings.loc[id, dataset.column +"_en"] = translation
        else:
            print("already done in ", i)

        listings.to_pickle(path)

In [120]:
# host about 
!python script.py

already done in  0
already done in  1
already done in  2
already done in  3
already done in  4
already done in  5
already done in  6
already done in  7
already done in  8
already done in  9
already done in  10
already done in  11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
Downloading: 100% 1.11k/1.11k [00:00<00:00, 817kB/s]
Downloading: 100% 286M/286M [00:06<00:00, 44.9MB/s]
Downloading: 100% 800k/800k [00:00<00:00, 2.13MB/s]
Downloading: 100% 787k/787k [00:00<00:00, 2.48MB/s]
Downloading: 100% 1.29M/1.29M [00:00<00:00, 3.44MB/s]
Downloading: 100% 42.0/42.0 [00:00<00:00, 35.4kB/s]
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147

In [124]:
translated_ha = pd.read_pickle("/content/drive/MyDrive/Colab/airbnb/data/translations/host_about")

In [125]:
translated_ha.index = translated_ha.host_about

In [132]:
ha_dict = translated_ha["host_about_en"].to_dict()

In [137]:
listings_mapped_translations = listings["host_about"].map(ha_dict)

In [138]:
listings["host_about_en"] = listings_mapped_translations

In [144]:
path = "/content/drive/MyDrive/Colab/airbnb/data/translations/listings"
listings.to_pickle(path)

In [145]:
listings["description_en"][500:550]

7197     The apartment is very close to two subway line...
9894     In the accommodation 1210 apartment is a compl...
5478     Nice flat for a wonderful stay in Vienna. Near...
8453     Perfect to explore Vienna. It is a private acc...
4548     This attic apartment offers a pleasant holiday...
8457     Newly furnished 2-room apartment with Viennese...
1849     Dear travellers! Welcome to a stylish home in ...
9889     In March 2020 this house was opened with units...
1852     Near the river! The Danube is across the road,...
8455     Our apartment is comfortably furnished and can...
8456     Vienna is a beautiful city that invites many v...
5472     Close to the Danube Canal, very bright, quiet ...
1856     Daughter 19 abroad - her room is vacated and w...
3754     Are you looking for a cozy and bright accommod...
9884     SCROLL DOWN FOR ENGLISH DESCRIPTION PLEASE! <b...
9883     This first-class 36 m2 design apartment is cen...
7203     Bright apartment in a prime location with holi.